In [1]:
import pandas as pd
import numpy as np

# 1. Data preprocessing

In [2]:
file_path = 'data.xlsx'

df = pd.read_excel(file_path)

In [3]:
df.head()

,Numéro d'application,Date d'application,Numero de publication,date de publication,CPC,IPC,claim,description
0,17167919,2017-04-25,3264223,2018-01-03,"['B23K26-361', 'H04M1-0266', 'B23K26-402', 'G0...","['C08J7-04', 'H04M1-02', 'G06F1-16', 'B23K26-3...","<!-- EPO <DP n=""18""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001..."
1,17166775,2017-04-18,3235458,2017-10-25,"['A61B17-68', 'A61B17-888', 'A61B17-8685', 'A6...","['A61B17-88', 'A61B17-68', 'A61B17-86']","<!-- EPO <DP n=""20""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">FIE..."
2,17166069,2017-04-11,3249094,2017-11-29,"['D06F58-206', 'D06F58-20']","['D06F58-20', 'D06F58-02']","<!-- EPO <DP n=""22""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001""><b>..."
3,18771363,2018-03-05,3605724,2020-02-05,"['H01P1-2138', 'H01P5-107', 'H01P1-2088', 'H01...","['H01P3-12', 'H01P1-213', 'H01P5-107']","<!-- EPO <DP n=""62""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">Tec..."
4,19881950,2019-11-01,3713245,2020-09-23,"['H04N21-4345', 'H04N21-4821', 'H04N21-4316']","['H04N21-482', 'H04N21-262']","<!-- EPO <DP n=""14""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001..."


check for missing values or duplicates

In [4]:
df.isna().sum()

Numéro d'application     0
Date d'application       0
Numero de publication    0
date de publication      0
CPC                      0
IPC                      0
claim                    0
description              0
dtype: int64

In [5]:
df.duplicated().sum()

0

In [6]:
df.shape

(50000, 8)

In [7]:
df.dtypes

Numéro d'application              int64
Date d'application       datetime64[ns]
Numero de publication             int64
date de publication      datetime64[ns]
CPC                              object
IPC                              object
claim                            object
description                      object
dtype: object

In [8]:
import re
import ast

def split_code(l):
    l = ast.literal_eval(l)
    splitted_code_list = []
    splitted_code = []
    for elt in l:
        elt = elt.split('-')
        letters = list(filter(None, re.split(r'[0-9]', elt[0])))
        digits = list(filter(None, re.split(r'[A-Z]', elt[0])))
        code = [sub[item] for item in range(len(digits))
                           for sub in [letters, digits]]
        
        code.append(elt[1])
        splitted_code.append(code)
        
        splitted_code_list = [list(elements) for elements in zip(*splitted_code)]
        
    #return splitted_code_list[0], splitted_code_list[1], splitted_code_list[2], splitted_code_list[3], splitted_code_list[4]
    return splitted_code_list
    
res = split_code("['B23K26-361', 'H04M1-0266','B23K26-402']")
print(res)

[['B', 'H', 'B'], ['23', '04', '23'], ['K', 'M', 'K'], ['26', '1', '26'], ['361', '0266', '402']]


In [9]:
cpc_df = df['CPC'].apply(split_code).apply(pd.Series)
cpc_df.columns = ['CPC_section','CPC_class','CPC_subclass','CPC_group','CPC_subgroup']

In [10]:
cpc_df.head()

,CPC_section,CPC_class,CPC_subclass,CPC_group,CPC_subgroup
0,"[B, H, B, G, G, G, G, B, B, B, B, G]","[23, 04, 23, 06, 06, 06, 02, 23, 23, 23, 23, 02]","[K, M, K, F, F, F, F, K, K, K, K, F]","[26, 1, 26, 1, 1, 1, 1, 2101, 2101, 2103, 2103...","[361, 0266, 402, 1626, 1656, 1637, 133308, 34,..."
1,"[A, A, A, A, A, A, A, A, A]","[61, 61, 61, 61, 61, 61, 61, 61, 61]","[B, B, B, B, B, B, B, B, B]","[17, 17, 17, 17, 17, 17, 17, 17, 2017]","[68, 888, 8685, 864, 8605, 8635, 863, 8891, 8655]"
2,"[D, D]","[06, 06]","[F, F]","[58, 58]","[206, 20]"
3,"[H, H, H, H, H]","[01, 01, 01, 01, 01]","[P, P, P, P, P]","[1, 5, 1, 5, 1]","[2138, 107, 2088, 182, 268]"
4,"[H, H, H]","[04, 04, 04]","[N, N, N]","[21, 21, 21]","[4345, 4821, 4316]"


In [11]:
df = df.join(cpc_df)

In [12]:
df.head()

,Numéro d'application,Date d'application,Numero de publication,date de publication,CPC,IPC,claim,description,CPC_section,CPC_class,CPC_subclass,CPC_group,CPC_subgroup
0,17167919,2017-04-25,3264223,2018-01-03,"['B23K26-361', 'H04M1-0266', 'B23K26-402', 'G0...","['C08J7-04', 'H04M1-02', 'G06F1-16', 'B23K26-3...","<!-- EPO <DP n=""18""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001...","[B, H, B, G, G, G, G, B, B, B, B, G]","[23, 04, 23, 06, 06, 06, 02, 23, 23, 23, 23, 02]","[K, M, K, F, F, F, F, K, K, K, K, F]","[26, 1, 26, 1, 1, 1, 1, 2101, 2101, 2103, 2103...","[361, 0266, 402, 1626, 1656, 1637, 133308, 34,..."
1,17166775,2017-04-18,3235458,2017-10-25,"['A61B17-68', 'A61B17-888', 'A61B17-8685', 'A6...","['A61B17-88', 'A61B17-68', 'A61B17-86']","<!-- EPO <DP n=""20""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">FIE...","[A, A, A, A, A, A, A, A, A]","[61, 61, 61, 61, 61, 61, 61, 61, 61]","[B, B, B, B, B, B, B, B, B]","[17, 17, 17, 17, 17, 17, 17, 17, 2017]","[68, 888, 8685, 864, 8605, 8635, 863, 8891, 8655]"
2,17166069,2017-04-11,3249094,2017-11-29,"['D06F58-206', 'D06F58-20']","['D06F58-20', 'D06F58-02']","<!-- EPO <DP n=""22""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001""><b>...","[D, D]","[06, 06]","[F, F]","[58, 58]","[206, 20]"
3,18771363,2018-03-05,3605724,2020-02-05,"['H01P1-2138', 'H01P5-107', 'H01P1-2088', 'H01...","['H01P3-12', 'H01P1-213', 'H01P5-107']","<!-- EPO <DP n=""62""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">Tec...","[H, H, H, H, H]","[01, 01, 01, 01, 01]","[P, P, P, P, P]","[1, 5, 1, 5, 1]","[2138, 107, 2088, 182, 268]"
4,19881950,2019-11-01,3713245,2020-09-23,"['H04N21-4345', 'H04N21-4821', 'H04N21-4316']","['H04N21-482', 'H04N21-262']","<!-- EPO <DP n=""14""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001...","[H, H, H]","[04, 04, 04]","[N, N, N]","[21, 21, 21]","[4345, 4821, 4316]"


In [13]:
res = split_code(df['IPC'].loc[0])
print(res)

[['C', 'H', 'G', 'B', 'B', 'B'], ['08', '04', '06', '23', '23', '23'], ['J', 'M', 'F', 'K', 'K', 'K'], ['7', '1', '1', '26', '26', '103'], ['04', '02', '16', '362', '402', '00']]


In [14]:
ipc_df = df['IPC'].apply(split_code).apply(pd.Series)
ipc_df.columns = ['IPC_section','IPC_class','IPC_subclass','IPC_group','IPC_subgroup']

C:\Users\kevmu\AppData\Local\Temp\ipykernel_16708\1327379969.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ipc_df = df['IPC'].apply(split_code).apply(pd.Series)


In [15]:
ipc_df.head()

,IPC_section,IPC_class,IPC_subclass,IPC_group,IPC_subgroup
0,"[C, H, G, B, B, B]","[08, 04, 06, 23, 23, 23]","[J, M, F, K, K, K]","[7, 1, 1, 26, 26, 103]","[04, 02, 16, 362, 402, 00]"
1,"[A, A, A]","[61, 61, 61]","[B, B, B]","[17, 17, 17]","[88, 68, 86]"
2,"[D, D]","[06, 06]","[F, F]","[58, 58]","[20, 02]"
3,"[H, H, H]","[01, 01, 01]","[P, P, P]","[3, 1, 5]","[12, 213, 107]"
4,"[H, H]","[04, 04]","[N, N]","[21, 21]","[482, 262]"


In [16]:
df = df.join(ipc_df)

In [17]:
df.head()

,Numéro d'application,Date d'application,Numero de publication,date de publication,CPC,IPC,claim,description,CPC_section,CPC_class,CPC_subclass,CPC_group,CPC_subgroup,IPC_section,IPC_class,IPC_subclass,IPC_group,IPC_subgroup
0,17167919,2017-04-25,3264223,2018-01-03,"['B23K26-361', 'H04M1-0266', 'B23K26-402', 'G0...","['C08J7-04', 'H04M1-02', 'G06F1-16', 'B23K26-3...","<!-- EPO <DP n=""18""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001...","[B, H, B, G, G, G, G, B, B, B, B, G]","[23, 04, 23, 06, 06, 06, 02, 23, 23, 23, 23, 02]","[K, M, K, F, F, F, F, K, K, K, K, F]","[26, 1, 26, 1, 1, 1, 1, 2101, 2101, 2103, 2103...","[361, 0266, 402, 1626, 1656, 1637, 133308, 34,...","[C, H, G, B, B, B]","[08, 04, 06, 23, 23, 23]","[J, M, F, K, K, K]","[7, 1, 1, 26, 26, 103]","[04, 02, 16, 362, 402, 00]"
1,17166775,2017-04-18,3235458,2017-10-25,"['A61B17-68', 'A61B17-888', 'A61B17-8685', 'A6...","['A61B17-88', 'A61B17-68', 'A61B17-86']","<!-- EPO <DP n=""20""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">FIE...","[A, A, A, A, A, A, A, A, A]","[61, 61, 61, 61, 61, 61, 61, 61, 61]","[B, B, B, B, B, B, B, B, B]","[17, 17, 17, 17, 17, 17, 17, 17, 2017]","[68, 888, 8685, 864, 8605, 8635, 863, 8891, 8655]","[A, A, A]","[61, 61, 61]","[B, B, B]","[17, 17, 17]","[88, 68, 86]"
2,17166069,2017-04-11,3249094,2017-11-29,"['D06F58-206', 'D06F58-20']","['D06F58-20', 'D06F58-02']","<!-- EPO <DP n=""22""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001""><b>...","[D, D]","[06, 06]","[F, F]","[58, 58]","[206, 20]","[D, D]","[06, 06]","[F, F]","[58, 58]","[20, 02]"
3,18771363,2018-03-05,3605724,2020-02-05,"['H01P1-2138', 'H01P5-107', 'H01P1-2088', 'H01...","['H01P3-12', 'H01P1-213', 'H01P5-107']","<!-- EPO <DP n=""62""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">Tec...","[H, H, H, H, H]","[01, 01, 01, 01, 01]","[P, P, P, P, P]","[1, 5, 1, 5, 1]","[2138, 107, 2088, 182, 268]","[H, H, H]","[01, 01, 01]","[P, P, P]","[3, 1, 5]","[12, 213, 107]"
4,19881950,2019-11-01,3713245,2020-09-23,"['H04N21-4345', 'H04N21-4821', 'H04N21-4316']","['H04N21-482', 'H04N21-262']","<!-- EPO <DP n=""14""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001...","[H, H, H]","[04, 04, 04]","[N, N, N]","[21, 21, 21]","[4345, 4821, 4316]","[H, H]","[04, 04]","[N, N]","[21, 21]","[482, 262]"


In [ ]:
def remove_html_tags(text):
    clean_text = re.sub('<.*?>', '', text)
    return clean_text

data['claim'] = data['claim'].apply(remove_html_tags)

data['claim'].head()